## Twitter sentiment analysis with pretrained embedding

In this notebook we will analyze the same dataset as in the sentAnalysis.ipynb notebook, except this time using a pretrained embedding from the GloVe model.

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.2'

In [2]:
import os

home_dir = os.getcwd()
fname = os.path.join(home_dir,'data/train_data.csv')

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
lines = lines[:-1]

sents = [0,0,0,0,0]
labels = []
texts = []
for line in lines:
    sent = int(line.split(',')[0])
    tweet = line.split(',')[1]
    sents[sent -1] +=1
    texts.append(tweet)
    labels.append(sent-1) # data is labeled 1-5; we shift by 1 so it starts with 0
 
print(header)
print(sents)
print(len(labels))
print(len(texts))

print("A sample tweet: " + texts[6] )
print("Has sentiment: " + str(labels[6]))

['sentiment', 'text']
5250
[76, 473, 3287, 1062, 352]
5250
5250
A sample tweet: Autonomous vehicles could reduce traffic fatalities by 90%...I'm in!
Has sentiment: 4


## Tokenize the data


In [45]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 20 # We cut the tweet after 20 words (most are shorter than this anyway)
training_samples = 4000
validation_samples = len(labels) - training_samples
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 10982 unique tokens.
Shape of data tensor: (5250, 20)
Shape of label tensor: (5250,)


In [46]:
y_train[0]

4

In [47]:
#one-hot encode the labels
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train,num_classes=5)
y_val = to_categorical(y_val,num_classes=5)

In [48]:
y_train[0]

array([ 0.,  0.,  0.,  0.,  1.])

### GloVe Embedding

Embedding can be downloaded from `https://nlp.stanford.edu/projects/glove/`. Its the 822 MB file named `glove.6B.zip`

In [4]:
glove_dir = '/Users/chrisheinrich/ml/nlp/embeddings/glove'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


Now we will build an embedding matrix that we can load into an `Embedding` layer. It must e a matrix of shape `(max_words, embedding_dim)`, where each entry `i` contains the vector for the word of index `i` in our reference word index (built during tokenization). The index `0` is only a placeholder

In [49]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    

### Define a model

In [50]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 100)           1000000   
_________________________________________________________________
flatten_7 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                64032     
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 165       
Total params: 1,064,197
Trainable params: 1,064,197
Non-trainable params: 0
_________________________________________________________________


### Load the GloVe embeddings in the model

In [51]:
model.layers[0].set_weights = ([embedding_matrix])
model.layers[0].trainable = False

In [52]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['acc'])


In [53]:
history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val,y_val))

Train on 4500 samples, validate on 750 samples
Epoch 1/10
4500/4500 [==============================] - 1s 117us/step - loss: 1.1783 - acc: 0.6204 - val_loss: 1.1270 - val_acc: 0.6240
Epoch 2/10
4500/4500 [==============================] - 0s 52us/step - loss: 1.0857 - acc: 0.6264 - val_loss: 1.0975 - val_acc: 0.6240
Epoch 3/10
4500/4500 [==============================] - 0s 53us/step - loss: 1.0514 - acc: 0.6264 - val_loss: 1.0895 - val_acc: 0.6240
Epoch 4/10
4500/4500 [==============================] - 0s 51us/step - loss: 1.0226 - acc: 0.6271 - val_loss: 1.0897 - val_acc: 0.6240
Epoch 5/10
4500/4500 [==============================] - 0s 53us/step - loss: 0.9904 - acc: 0.6298 - val_loss: 1.0951 - val_acc: 0.6227
Epoch 6/10
4500/4500 [==============================] - 0s 53us/step - loss: 0.9573 - acc: 0.6382 - val_loss: 1.0971 - val_acc: 0.6240
Epoch 7/10
4500/4500 [==============================] - 0s 52us/step - loss: 0.9225 - acc: 0.6493 - val_loss: 1.1051 - val_acc: 0.6200
Epoch 8

The performance is comparable to what we obtained without using a pre-trained word embedding. It seems that there is just not enough data to learn a language model that is able t